5 Oct. 2024

Using image labels to train object detection model on lego friends

References:
- https://cocodataset.org/#home (Coco model)
- https://www.robots.ox.ac.uk/~vgg/software/via/via.html (Labeling online software)
- https://www.kaggle.com/code/hmendonca/airbus-mask-rcnn-and-coco-transfer-learning (Coco transfer learning example)
- https://github.com/ultralytics/yolov5/issues/980 (Coco additional classes)
- ChatGPT (questions throughout the process)

# 1. Parse Json file for data we'll use

In [1]:
import json

In [8]:
# Loading Json file
with open('/content/drive/MyDrive/Colab Notebooks/via_project_24Sep2024_18h40m_json.json') as f:
  data = json.load(f)

# data.keys()

Example format for 1 image:

"lego-00.11.20-09.18.2024.jpg69105":{"filename":"lego-00.11.20-09.18.2024.jpg","size":69105,"regions":[{"shape_attributes":{"name":"rect","x":215,"y":200,"width":98,"height":122},"region_attributes":{"Name":"Lego Friend"}}],"file_attributes":{}}

In [9]:
images = []
annotations = []

# Parsing through json file and identifying data stored for each image
for key, value in data.items():
  filename = value['filename']

  # There's only one region per image, but we'll use a for loop just to be sure
  for region in value['regions']:
    shape_attributes = region['shape_attributes']
    shape = shape_attributes['name']
    x = shape_attributes['x']
    y = shape_attributes['y']
    width = shape_attributes['width']
    height = shape_attributes['height']

    obj = region['region_attributes']['Name']

    # Calculate the bounding box coordinates
    xmin = x
    ymin = y
    xmax = x + width
    ymax = y + height

    images.append(filename)
    annotations.append({'bbox': [xmin, ymin, xmax, ymax],
                        'obj': obj})

In [10]:
print(images)
print()
print(annotations)

['lego-00.11.20-09.18.2024.jpg', 'lego-01.06.20-09.18.2024.jpg', 'lego-02.15.20-09.18.2024.jpg', 'lego-03.10.20-09.18.2024.jpg', 'lego-03.10.20-09.18.2024.jpg', 'lego-04.13.20-09.18.2024.jpg', 'lego-05.11.20-09.18.2024.jpg', 'lego-05.14.20-09.18.2024.jpg', 'lego-06.07.20-09.18.2024.jpg', 'lego-06.07.20-09.18.2024.jpg', 'lego-07.08.20-09.18.2024.jpg', 'lego-07.08.20-09.18.2024.jpg', 'lego-07.12.20-09.18.2024.jpg', 'lego-07.12.20-09.18.2024.jpg', 'lego-07.17.20-09.18.2024.jpg', 'lego-07.17.20-09.18.2024.jpg', 'lego-08.06.20-09.18.2024.jpg', 'lego-09.05.20-09.18.2024.jpg', 'lego-09.05.20-09.18.2024.jpg', 'lego-09.09.20-09.18.2024.jpg', 'lego-09.16.20-09.18.2024.jpg', 'lego-09.16.20-09.18.2024.jpg', 'lego-11.04.20-09.18.2024.jpg', 'lego-12.17.20-09.18.2024.jpg', 'lego-13.02.20-09.18.2024.jpg', 'lego-13.14.20-09.18.2024.jpg', 'lego-13.14.20-09.18.2024.jpg', 'lego-14.05.20-09.18.2024.jpg', 'lego-14.11.20-09.18.2024.jpg', 'lego-15.15.20-09.18.2024.jpg', 'lego-16.16.20-09.18.2024.jpg', 'lego-1

In [11]:
print(f"Number of images in list: {len(images)}")
print(f"Number of annotations in list: {len(annotations)}")

Number of images in list: 130
Number of annotations in list: 130


# 2. Preparing our Dataset
- Splitting data into training and testing sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_images, validate_images, train_annotations, validate_annotations = train_test_split(images, annotations, test_size=0.2, random_state=1)

In [14]:
print(f"Number of training images: {len(train_images)} and number of training annotations: {len(train_annotations)}")
print(f"Number of validation images: {len(validate_images)} and number of validation annotations: {len(validate_annotations)}")

Number of training images: 104 and number of training annotations: 104
Number of validation images: 26 and number of validation annotations: 26


# 3. Convert Annotations to Tensorflow Format
- Want to use tf.data.Dataset, so we need to reformat our data to fit into this

In [ ]:
import tensorflow as tf

In [16]:
def create_tf_data(image, annotations):
  """
  Takes the image and the annotations parsed from json file and converts it to an "example,"
  which is what tf.data.Dataset needs to be fed. We're serializing the data so we can put it into
  tf.data.Dataset and thus use it later.
  """
  image_path = f'/content/drive/MyDrive/lego_men_photos/{image}'
  image_string = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(image_string, channels=3)

  # Identifying the bounding box and object name in annotations
  bbx = annotations['bbox']
  obj_name = annotations['obj']

  # Creating tf.train.Example
  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_string.numpy()])),
      # We appended data into bbox as 'bbox': [xmin, ymin, xmax, ymax], so we must label it in this order, too
      'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[bbx[0]])),
      'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[bbx[1]])),
      'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[bbx[2]])),
      'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[bbx[3]])),
      'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[obj_name.encode('utf8')])),
  }))

  return tf_example


# Using list comprehension to run all training data (images and annotations) through creat_df_data function
train_tfrecords = [create_tf_data(train_images[i], train_annotations[i]) for i in range(len(train_images))]

# Using list comprehension to run all validation data through, too
validate_tfrecords = [create_tf_data(validate_images[i], validate_annotations[i]) for i in range(len(validate_images))]

# Write training examples to TFRecord files
with tf.io.TFRecordWriter('training.record')as writer:
  for example in train_tfrecords:
    writer.write(example.SerializeToString())

# Write validation examples to TFRecord files
with tf.io.TFRecordWriter('validation.record') as writer:
  for example in validate_tfrecords:
    writer.write(example.SerializeToString())


### Coding Notes
- It's considered best practice to have a comma at the end of a value-key pair, since it makes it easier to add / remove lines later manually and with version control
- We use value=[bbx[3]] and not value=bbx[3] because "Tensorflow FloatList expects its input to be a list," according to ChatGPT.

Now we have the training and validation records to use later with Tensorflow.

# 4. Model Training
(summary of next instructions from ChatGPT)
- Using TensorFlow's Object Detection API
- Must choose a model (here we'll use Coco)
- Edit config file
  - Set paths for training and validation files (we just made these)
  -   Adjust the number of classes (here we have 1: "Lego Friend")
  - Set batch size, learning rate, ect.

## Download Tensorflow and Tensorflow Object Detection API
- Using Colab like a terminal now
  - Use a lot of "!"'s to signify that we're typing bash/command terminal commands

https://github.com/tensorflow/models.git = TensorFlow Model Garden (can pick models out to use from here)
- We'll be using the API

In [66]:
# Install tensorflow
!pip install tensorflow==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 771.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: g

In [19]:
# Cloning TensorFlow models repo from github (including API)
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


## Protobuf Installation & Compilation

Reference for a lot of the code here:
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

In [20]:
# Installing the obj. detection API
!apt-get install protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [22]:
# Changing directory to models/research && downloading and compiling the protobuf libraries
!cd models/research && protoc object_detection/protos/*.proto --python_out=.
# !cp object_detection/packages/tf2/setup.py .
# !python -m pip install .

## COCO API Installation
- Still following instructions from above link
- COCO API: https://github.com/cocodataset/cocoapi.git

In [24]:
# Downloading cocoapi
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975 (from 1)
Receiving objects: 100% (975/975), 11.72 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [25]:
# Making the cocoapi repo
!cd cocoapi/PythonAPI && make

python setup.py build_ext --inplace
running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-310
creating build/temp.linux-x86_64-cpython-310/pycocotools
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/numpy/core/include -I../common -I/usr/include/python3.10 -c ../common/maskApi.c -o build/temp.linux-x86_64-cpython-310/../common/maskApi.o -Wno-cpp -Wno-unused-funct

In [27]:
# !cp -r pycocotools /content/models/research/ # Error...
!cp -r cocoapi /content/models/research/

## Install Object Detection API
- Still following instructions in above link

In [31]:
# !cd models/research && protoc object_detection/protos/*.proto --python_out=.
# !cp object_detection/packages/tf2/setup.py .
# !python -m pip install .
!cd models/research && cp object_detection/packages/tf2/setup.py . && python -m pip install .

#!python -m pip install . # Doesn't like this because it forgets we're in models/research! Put in above line!

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 6.4 MB/s eta 0:00

In [32]:
# Add libraries to python path
import sys
sys.path.append('models/research')
sys.path.append('models/research/slim')

## Reaccess Dataset Preparation
- What we already wrote to the record files

In [33]:
training_record = '/content/training.record'
validation_record = '/content/validation.record'

## Configure the Model
- Download pretrained model: Coco SSD MobileNet V2 for now
- Edit configuration file (.config)
  - Paths for training and validation datasets
  - Adjust the number of classes / batch size / ect.

In [35]:
# Downloading the Model from Tensorflow and unpacking zip file
!wget https://storage.googleapis.com/download.tensorflow.org/models/tflite/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip

--2024-10-05 18:17:46--  https://storage.googleapis.com/download.tensorflow.org/models/tflite/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.183.207, 142.250.125.207, 142.251.184.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.183.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2807218 (2.7M) [application/zip]
Saving to: ‘coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip’

coco_ssd_mobilenet_ 100%[===================>]   2.68M  --.-KB/s    in 0.02s   

2024-10-05 18:17:46 (129 MB/s) - ‘coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip’ saved [2807218/2807218]



In [36]:
# Unzipping file
!unzip /content/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip

Archive:  /content/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29.zip
  inflating: detect.tflite           
  inflating: labelmap.txt            


In [37]:
# Editing Configuration file
!cp coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/pipeline.config /content/pipeline.config

cp: cannot stat 'coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/pipeline.config': No such file or directory


## Reconfiguring the model
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

In [51]:
!pip install tensorflow_hub

In [52]:
import tensorflow as tf
import tensorflow_hub as hub

# Loading model with tensorflow hub because of http issues...
model_url = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
model = hub.load(model_url)

- Tensorflow hub doesn't download the model into the files or directory but instead straight into memory...
- I want to see if I can save it as a tangible file so I don't have to load it every time

In [50]:
#!wget -O ssd_resnet50_v1_fpn_640x640.tar.gz \
#http://download.tensorflow.org/models/ssd_resnet50_v1_fpn_640x640.tar.gz
# Doesn't like the link...403 error (security???)

--2024-10-05 18:59:23--  http://download.tensorflow.org/models/ssd_resnet50_v1_fpn_640x640.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.206.207, 142.251.183.207, 142.250.125.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.206.207|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-10-05 18:59:23 ERROR 403: Forbidden.



In [55]:
# Saving model to disk for future use
import os

# Create directory to save model to
save_directory = 'content/faster_rcnn_model'
os.makedirs(save_directory, exist_ok=True)

# Saving the model
tf.saved_model.save(model, save_directory)

- Saved in content/faster_rcnn_model
- If want to load model later:


```
load_model = tf.saved_model.load(save_dir)
```
(From ChatGPT)


## Reconfiguring once more (Manually downloading the model this time)

In [57]:
!cd /content/drive/MyDrive/ && tar -xvf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

ssd_mobilenet_v2_320x320_coco17_tpu-8/
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.index


### Configuring the Training Pipeline
- Following link instructions still

In [58]:
# Copy the pipeline file for this model to the content folder
!cd /content/drive/MyDrive/ssd_mobilenet_v2_320x320_coco17_tpu-8 && cp pipeline.config /content/pipeline.config
# !cp /pipeline.config /content/pipeline.config

Now we need to change the actual file...
1. Opening file in Google Colab
2. [line 3] num_classes = 1 (We only have "Lego Friend")
3. [line 138] batch_size = 8 (We can increase/decrease based on memory space later)
4. [line 162] fine_tune_checkpoint: "/content/drive/MyDrive/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0"
  - DON'T INCLUDE FILE EXTENTIONS
  - Tensorflow will go through and look at all files with the ckpt-0 file name later
    - More flexibility and abiding by convention
    - Explained by ChatGPT
5. [line 168] fine_tune_checkpoint_type: "detection"
  - We want it to be training the whole detection model
6. [line 172] label_map_path: "/content/drive/MyDrive/label_map_legofriend.pbtxt"
7. [line 174] input_path: "/content/validation.record"
8. [line 182] label_map_path: "/content/drive/MyDrive/label_map_legofriend.pbtxt"
9. [line 186] input_path: "/content/validation.record"

### Making a label_map.pbtxt file
- We only have one class (Lego Friend), so we only need one item in our label class
- This is what our label map file looks like (Created with ChatGPT's help ['cause I didn't understand what the heck this was before I asked it, lol])


```
item {
  id: 1
  name: 'Lego Friend'
}
```



# Training the Model
Reference: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html


1. Want to use the /content/models/research/object_detection/model_main_tf2.py script from TensorFlow to train our model
  - Copy into our training folder (which for me is the /content file)
2. From the directory this file is in (/content), run the command:
```
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config
```
  - WAIT for it to run...no touchy!

  
  ![no touchy](https://meme-generator.com/wp-content/uploads/mememe/2020/03/mememe_d75ef82eed2390ac48ea8a2a472f4a59-1.jpg)



### The model_dir here
- ALL INFO ABOUT MODEL USED IS IN PIPELINE FILE
- The model_dir here means where the training will store the checkpoints
- Create new directory for this: /content/drive/MyDrive/model_dir_1

In [61]:
# Copying file into workspace
!cp /content/models/research/object_detection/model_main_tf2.py /content/drive/MyDrive/model_main_tf2.py

In [62]:
# Running the long training prompt and filling in values
# python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config
!cd /content/drive/MyDrive/ && python model_main_tf2.py --model_dir=/content/drive/MyDrive/model_dir_1 --pipeline_config_path=pipeline.config --alsologtostderr
# Added --alsologtostderr because ChatGPT said this would show logs in the terminal

2024-10-05 20:58:39.226202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 20:58:39.248216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-05 20:58:39.254606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 20:58:40.756116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/drive/MyDrive/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "/usr/local/lib/python3.10/dist-packages/object_detection/model_lib_v2.py", line 

# Exporting Trained Model
Instructions from ChatGPT

```
!python models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/pipeline.config \
    --trained_checkpoint_dir=/content/model_dir \
    --output_directory=/content/exported-model

```
- According to my understanding from what ChatGPT is saying, I shouldn't have to create any of these directories; training will create them
- Training will create checkpoints throughout training, and that's what it's referencing with trained_checkpoint_dir


In [ ]:
# Exporting, changed pipeline path
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/MyDrive/pipeline.config \
    --trained_checkpoint_dir=/content/drive/MyDrive/model_dir_1 \
    --output_directory=/content/drive/MyDrive/exported-model

# Converting to TensorFlow Lite
Instructions from ChatGPT


```
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/exported-model/saved_model')
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

```



# Downloading the Model
Instructions from ChatGPT


```
from google.colab import files

files.download('model.tflite')

```



Next steps according to ChatGPT

    Edit the Configuration File: You need to edit the .config file corresponding to the model you downloaded. You can use sed or any text editor to make the necessary changes. Here's an example:

python

!cp ssd_mobilenet_v2_coco_2018_03_29/pipeline.config /content/pipeline.config

# Edit the pipeline.config file to set paths and hyperparameters
!sed -i 's|PATH_TO_BE_CONFIGURED|/content|g' /content/pipeline.config

Make sure to set the paths for your training and validation datasets and adjust the number of classes, batch size, etc.
Step 5: Train the Model

Run the training script using the following command:

python

!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/pipeline.config \
    --model_dir=/content/model_dir \
    --alsologtostderr \
    --num_steps=5000  # Adjust the number of steps as needed

Step 6: Export the Trained Model

Once training is complete, you can export the trained model to a TensorFlow SavedModel format or directly to TensorFlow Lite:

python

!python models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/pipeline.config \
    --trained_checkpoint_dir=/content/model_dir \
    --output_directory=/content/exported-model

Step 7: Convert to TensorFlow Lite (Optional)

If you want to convert the model to TensorFlow Lite:

python

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/exported-model/saved_model')
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Step 8: Download the Model

Finally, you can download your model or any files created during the process:

python

from google.colab import files

files.download('model.tflite')